In [5]:
from bs4 import BeautifulSoup
import selenium
from selenium.webdriver.common.by import By
import pandas as pd
import time
from pprint import pprint
import re
from selenium import webdriver


driver = webdriver.Chrome()
# https://www.google.com/maps/search/boris+lum%C3%A9/@48.8872009,2.3321052,17z/data=!3m1!4b1
url = 'https://www.google.com/maps/search/URSSAF/@48.8538201,2.1341141,11z/data=!3m1!4b1'
driver.get(url)
button_refuse_cookie = driver.find_element(By.CLASS_NAME, 'VfPpkd-LgbsSe.VfPpkd-LgbsSe-OWXEXe-k8QpJ.VfPpkd-LgbsSe-OWXEXe-dgl2Hf.nCP5yc.AjY5Oe.DuMIQc.LQeN7.Nc7WLe').click()
time.sleep(1)

scrollable_div = driver.find_elements(By.CLASS_NAME,'Nv2PK.THOPZb')
# scrollable_div = driver.find_elements(By.CLASS_NAME, 'm6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd')[1]

# while not driver.find_elements(By.CLASS_NAME, 'HlvSq'):
#     driver.execute_script(
#         'arguments[0].scrollTop = arguments[0].scrollHeight',
#         scrollable_div
#     )
#     time.sleep(2)
        

for i in scrollable_div[:2]:

    # clique sur les services
    element_click = i.find_element(By.CLASS_NAME,'hfpxzc').click()
    time.sleep(2)

    # Coordonnées du service
    coordonnee = driver.find_elements(By.CLASS_NAME,'Io6YTe.fontBodyMedium')
    time.sleep(3)

    # Localisation du service
    localisation = coordonnee[0].text

    # Telephone du service
    telephone = coordonnee[2].text

    # Clique sur le bouton avis
    nom = driver.find_elements(By.CLASS_NAME,'hh2c6')[1].click()
    time.sleep(2)

    # Prendre le nombre total d'avis
    notre_avis = driver.find_element(By.CLASS_NAME, 'PPCwl')
    notre_avis = notre_avis.text
    chaine_de_caracteres = notre_avis.replace('\u202f', '')
    match = re.search(r'(\d+)\s*avis', chaine_de_caracteres)
    if match:
        nombre_avis = int(match.group(1))
        # print(nombre_avis)
    else:
        print("Aucun avis trouvé.")
        

    # Scroller mes commentaires jusqu'en bas
    scrollable_div = driver.find_elements(By.CLASS_NAME,'m6QErb.DxyBCb.kA9KIf.dS8AEf')[2]

    for i in range(nombre_avis//8):
        driver.execute_script(
        'arguments[0].scrollTop = arguments[0].scrollHeight',
        scrollable_div
        )
        # Cliquer sur le bouton lire plus des commentaires
        button_as_plus = driver.find_elements(By.CLASS_NAME,'w8nwRe.kyuRq')
        for i in button_as_plus:
            i.click()
        time.sleep(1)


    # S'arreter quand il y'a plus de bouton partager
    # old_freq = 0
    # freq = scrollable_div.text.count('Partager\n')
    # if freq == old_freq:
    #     break
    # old_freq = freq


    # commentaires service
    comments_list = driver.find_elements(By.CLASS_NAME,'wiI7pd')
    comments_list = [i.text.replace('\n', '').replace('"', '') for i in comments_list]


    # Nom service    
    service = driver.find_element(By.CLASS_NAME,'iD2gKb.W1neJ').text
    services_list = []
    services_list = [service for i in range(len(comments_list))]
    
    
    # Date de publication
    dates_list = driver.find_elements(By.CLASS_NAME,'rsqaWe')
    dates_list = [dates_list[i].text for i, comment in enumerate(comments_list) if comment and len(dates_list) > i]


    # Nom de l'auteur de la publication
    nom_list = driver.find_elements(By.CLASS_NAME,'d4r55')
    nom_list = [nom_list[i].text for i, comment in enumerate(comments_list) if comment and len(nom_list) > i]
    

    # localisation du service
    localisation_list = [localisation for i in range(len(comments_list))]
    
    
    # Telephone du service
    telephone_list = [telephone for i in range(len(comments_list))]
    
    
    # Nombre d'étoile pour chaque commentaire
    # # etoile_list = []
    # etoile = driver.find_elements(By.CLASS_NAME,'hCCjke.vzX5Ic')
    # etoile_list = [comment.get_attribute() for comment in comments_list]
        
        
    # ****Je vais créer le dictionnaire au début et faire append à chaque fois pour avoir un seul dataframe
    dict_text = {
        'Services' : service,
        'Telephone' : telephone_list,
        'Localisation' : localisation,
        'Auteur' : nom_list,
        'Commentaires': comments_list,
        'Date' : dates_list
        # 'etoile' : etoile_list
    }


    # ****faire sortir le code du for et générer un seul dataframe global
    # Créer un dataframe  pour chaque service
    df = pd.DataFrame(dict_text)
    df = df.drop_duplicates().reset_index(drop=True)
    df.to_csv(f'../Data/{service} {localisation}.csv')

    # time.sleep(2)
